In [1]:
import sys
from pathlib import Path

SRC = Path().resolve() / "src"
if str(SRC) not in sys.path:
    sys.path.append(str(SRC))

%load_ext autoreload
%autoreload 2

In [ ]:
dataset_usage = 1.0
dataset_regenerate = False

seq_size = 4 # число токеном в скользящем окне датасета
seq_stride = 1

emb_dim=256
hidden_dim=512
num_layers=2
dropout_p=0.3
learning_rate=1e-3
weight_decay=0.01

model_train = True
model_load_path = "models/next_token_8_250820_184932.pth"

In [3]:
from data_utils import process_dataset, read_splits, truncate

if dataset_regenerate:
    process_dataset("data/raw_dataset.csv")

train_df, val_df, test_df = read_splits("data/dataset_processed.csv")

train_texts = truncate(list(train_df['text']), dataset_usage)
val_texsts = truncate(list(val_df['text']), dataset_usage)

print(len(train_texts))

1280000


In [4]:
from transformers import BertTokenizerFast
from tokenizer_utils import resolve_eos_token_id

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

eos_token_id = resolve_eos_token_id(tokenizer)

print("eos_token_id:", eos_token_id)

eos_token_id: 102


In [5]:

from torch.utils.data import DataLoader
from next_token_dataset import NextTokenDataset

train_dataset = NextTokenDataset(
    train_texts, 
    tokenizer, 
    eos_id=eos_token_id, 
    seq_size=seq_size, 
    stride=seq_stride
)

val_dataset = NextTokenDataset(
    val_texsts,
    tokenizer,
    eos_id=eos_token_id,
    seq_size=seq_size,
    stride=seq_stride
)

print(f"Train samples count: {len(train_dataset)}")
print(f"Val samples count: {len(val_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128)

100%|██████████| 160000/160000 [00:06<00:00, 24009.01it/s]

Train samples count: 4158366
Val samples count: 517103


In [6]:
from lstm_model import NextTokenLSTM
from device_utils import get_device

import torch
import torch.nn as nn

device = get_device()
print("Using device:", device)

model = NextTokenLSTM(
    vocab_size=tokenizer.vocab_size,
    emb_dim=emb_dim,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    dropout_p=dropout_p,
)

Using device: mps


In [7]:
from lstm_train import train_next_token
from datetime import datetime

if model_train:
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()   

    model = train_next_token(
        model,
        optimizer=optimizer,
        criterion=criterion,
        train_loader=train_loader,
        val_loader=val_loader,
        device=device,
        epochs=5
    )

    timestamp = datetime.now().strftime("%y%m%d_%H%M%S")
    torch.save(model.state_dict(), f"models/next_token_{seq_size}_{timestamp}.pth")
else:
    state = torch.load(model_load_path, map_location=device)
    model.load_state_dict(state)

100%|██████████| 4040/4040 [01:47<00:00, 37.49it/s]


Epoch 01 | Train Loss: 4.9434 | Val Loss: 4.8705 | Val PPL: 130.384 | Val Token Acc: 21.49%


Epoch: 2:  24%|██▎       | 7670/32488 [10:15<33:11, 12.46it/s]


KeyboardInterrupt: 

In [26]:
from autocomplete import autocomplete_text

comp_text = autocomplete_text(
    model=model, 
    tokenizer=tokenizer,
    eos_id=eos_token_id,
    seq_size=seq_size,
    text="history about",
)
comp_text

'the former quot the quot'